In [42]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [43]:
url_inicial = 'https://polcul.xoc.uam.mx/index.php/polcul/issue/archive/3'
url_root = 'https://polcul.xoc.uam.mx/index.php/polcul/issue/archive/3'
r=requests.get(url_inicial)

In [44]:
soup = BeautifulSoup(r.text, 'html.parser')

In [45]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://polcul.xoc.uam.mx/index.php/polcul/issue/view/7',
 'https://polcul.xoc.uam.mx/index.php/polcul/issue/view/6',
 'https://polcul.xoc.uam.mx/index.php/polcul/issue/view/5',
 'https://polcul.xoc.uam.mx/index.php/polcul/issue/view/4',
 'https://polcul.xoc.uam.mx/index.php/polcul/issue/view/3',
 'https://polcul.xoc.uam.mx/index.php/polcul/issue/view/2',
 'https://polcul.xoc.uam.mx/index.php/polcul/issue/view/1']

In [46]:
def get_url_items(sopa, url): 
    box = soup.find('ul', class_='issues_archive')
    volumen=box.findAll('div', class_='obj_issue_summary')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [47]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina https://polcul.xoc.uam.mx/index.php/polcul/issue/archive/3


In [48]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

7

In [49]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [50]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Política y Cultura'
    a='V. Ciencias Sociales'
    tem='Artes y Humanidades'
    url='http://polcul.xoc.uam.mx/'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h2', id='issue-number').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [51]:
list_scraper=list_scraper[0:25]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6


In [52]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Política y Cultura,V. Ciencias Sociales,Artes y Humanidades,No. 7: Políticas Públicas y Sociedad,,http://polcul.xoc.uam.mx/,https://polcul.xoc.uam.mx/index.php/polcul/art...
1,Política y Cultura,V. Ciencias Sociales,Artes y Humanidades,No. 6: Cultura de las mujeres,,http://polcul.xoc.uam.mx/,https://polcul.xoc.uam.mx/index.php/polcul/art...
2,Política y Cultura,V. Ciencias Sociales,Artes y Humanidades,No. 5: Sistema político mexicano,,http://polcul.xoc.uam.mx/,https://polcul.xoc.uam.mx/index.php/polcul/art...
3,Política y Cultura,V. Ciencias Sociales,Artes y Humanidades,"No. 4: Imágenes, representaciones y subjetividad",,http://polcul.xoc.uam.mx/,https://polcul.xoc.uam.mx/index.php/polcul/art...
4,Política y Cultura,V. Ciencias Sociales,Artes y Humanidades,No. 3: Solidaridad o competencia,,http://polcul.xoc.uam.mx/,https://polcul.xoc.uam.mx/index.php/polcul/art...
5,Política y Cultura,V. Ciencias Sociales,Artes y Humanidades,No. 2: Configuraciones del mundo actual,,http://polcul.xoc.uam.mx/,https://polcul.xoc.uam.mx/index.php/polcul/art...
6,Política y Cultura,V. Ciencias Sociales,Artes y Humanidades,No. 1: Mujeres y Política,,http://polcul.xoc.uam.mx/,https://polcul.xoc.uam.mx/index.php/polcul/art...


In [53]:
df_catalogo.to_csv('Revista173.03.csv', index=False)